## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

#### 1. Inicializar y cargar el contexto spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext # de foto de Michelle
sc = SparkContext.getOrCreate() # de foto de Michelle
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql import *
from pyspark.sql import SQLContext, functions as F, Row
from pyspark.sql.types import *
from pyspark.sql.types import StringType, IntegerType

Importamos los datos

In [ ]:
data_file = "indicadores_deuda.csv"

In [ ]:
deuda_raw = sc.textFile(data_file)

Separamos por comas

In [ ]:
deuda_parsed = deuda_raw.map(lambda x: x.split(","))

Comprobamos

In [ ]:
deuda_parsed.take(2)

[['country_name',
  'country_code',
  'DT.AMT.BLAT.CD',
  'DT.AMT.DLXF.CD',
  'DT.AMT.DPNG.CD',
  'DT.AMT.MLAT.CD',
  'DT.AMT.OFFT.CD',
  'DT.AMT.PBND.CD',
  'DT.AMT.PCBK.CD',
  'DT.AMT.PROP.CD',
  'DT.AMT.PRVT.CD',
  'DT.DIS.BLAT.CD',
  'DT.DIS.DLXF.CD',
  'DT.DIS.MLAT.CD',
  'DT.DIS.OFFT.CD',
  'DT.DIS.PCBK.CD',
  'DT.DIS.PROP.CD',
  'DT.DIS.PRVT.CD',
  'DT.INT.BLAT.CD',
  'DT.INT.DLXF.CD',
  'DT.INT.DPNG.CD',
  'DT.INT.MLAT.CD',
  'DT.INT.OFFT.CD',
  'DT.INT.PBND.CD',
  'DT.INT.PCBK.CD',
  'DT.INT.PROP.CD',
  'DT.INT.PRVT.CD'],
 ['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  '']]

Importamos el csv de deuda con los nombres cambiados para no tener problemas posteriormente por los puntos de los nombres de las columnas

In [ ]:
import pandas as pd
deuda = pd.read_csv('indicadores_deuda.csv', sep = ",", names = ['country_name', 'country_code','DT_AMT_BLAT_CD','DT_AMT_DLXF_CD','DT_AMT_DPNG_CD','DT_AMT_MLAT_CD',
  'DT_AMT_OFFT_CD',
  'DT_AMT_PBND_CD',
  'DT_AMT_PCBK_CD',
  'DT_AMT_PROP_CD',
  'DT_AMT_PRVT_CD',
  'DT_DIS_BLAT_CD',
  'DT_DIS_DLXF_CD',
  'DT_DIS_MLAT_CD',
  'DT_DIS_OFFT_CD',
  'DT_DIS_PCBK_CD',
  'DT_DIS_PROP_CD',
  'DT_DIS_PRVT_CD',
  'DT_INT_BLAT_CD',
  'DT_INT_DLXF_CD',
  'DT_INT_DPNG_CD',
  'DT_INT_MLAT_CD',
  'DT_INT_OFFT_CD',
  'DT_INT_PBND_CD',
  'DT_INT_PCBK_CD',
  'DT_INT_PROP_CD',
  'DT_INT_PRVT_CD'],
  skiprows = 1 ) # skip la fila del header ya que lo nombramos manualmente

Convertimos en dataframe de sql

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
deuda_df = sqlContext.createDataFrame(deuda)
deuda_df.show(10)

+------------+------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+
|country_name|country_code|DT_AMT_BLAT_CD|  DT_AMT_DLXF_CD|DT_AMT_DPNG_CD|DT_AMT_MLAT_CD|DT_AMT_OFFT_CD|DT_AMT_PBND_CD|DT_AMT_PCBK_CD|DT_AMT_PROP_CD|DT_AMT_PRVT_CD|DT_DIS_BLAT_CD|  DT_DIS_DLXF_CD|DT_DIS_MLAT_CD|DT_DIS_OFFT_CD|DT_DIS_PCBK_CD|DT_DIS_PROP_CD|DT_DIS_PRVT_CD|DT_INT_BLAT_CD|DT_INT_DLXF_CD|DT_INT_DPNG_CD|DT_INT_MLAT_CD|DT_INT_OFFT_CD|DT_INT_PBND_CD|DT_INT_PCBK_CD|DT_INT_PROP_CD|DT_INT_PRVT_CD|
+------------+------------+--------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------

Importamos el dataset de desarrollo

In [ ]:
desarrollo = pd.read_csv('indicadores_desarrollo.csv', sep = ",", names = ["country_name", "country_code", "anho", "GC_DOD_TOTL_CN", 
                                                                           "GC_DOD_TOTL_GD_ZS", "SP_DYN_CBRT_IN"], skiprows = 1 )

In [ ]:
desarrollo_df = sqlContext.createDataFrame(desarrollo)
desarrollo_df.show(10)

+------------+------------+-------------+---------------+-------------------+------------------+
|country_name|country_code|         anho| GC_DOD_TOTL_CN|  GC_DOD_TOTL_GD_ZS|    SP_DYN_CBRT_IN|
+------------+------------+-------------+---------------+-------------------+------------------+
|     Albania|         ALB|2014 [YR2014]|   1.0230408E12|  73.32024691984292|            12.259|
|     Albania|         ALB|2015 [YR2015]|  1.14550094E12|   79.8644448747259|12.197000000000001|
|     Albania|         ALB|2016 [YR2016]|  1.18881449E12|  80.73557483969229|             12.08|
|   Australia|         AUS|2014 [YR2014]|        6.747E8|  0.042207528166503|              13.2|
|   Australia|         AUS|2015 [YR2015]|      7.62718E8| 0.0469480198522591|              12.9|
|   Australia|         AUS|2016 [YR2016]|      9.07555E8|0.05464848252016901|              12.9|
| The Bahamas|         BHS|2014 [YR2014]|      5.15842E9|   47.2672793747079|             14.34|
| The Bahamas|         BHS|201

##### 2. Número de países distintos en cada dataset. Coinciden?

In [132]:
deuda_df.select("country_name").distinct().count()

124

In [133]:
desarrollo_df.select("country_name").distinct().count()

52

##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

In [135]:
deuda_df.groupby('country_name').agg(F.sum('DT_AMT_MLAT_CD')).show() # si queremos mostrar todos sería .show(124)

+--------------------+-------------------+
|        country_name|sum(DT_AMT_MLAT_CD)|
+--------------------+-------------------+
|          South Asia|     7.8517399295E9|
|                Chad|       9.57387095E7|
|            Paraguay|      2.076620309E8|
|    Congo, Dem. Rep.|       1.43105462E8|
|             Senegal|      1.704379291E8|
|          Cabo Verde|       3.99701249E7|
|Least developed c...|     4.0842755242E9|
|      Macedonia, FYR|      1.859528672E8|
|              Guyana|       3.46518065E7|
|             Eritrea|       1.54824754E7|
|         Philippines|      7.390564273E8|
|            Djibouti|       3.42549092E7|
|               Tonga|          2791253.2|
|                Fiji|          9082442.6|
|              Turkey|      2.932698678E9|
|              Malawi|       3.56598314E7|
|             Comoros|          2182053.1|
|         Afghanistan|        3.9107845E7|
|            Cambodia|       7.23070018E7|
|              Jordan|      2.704987155E8|
+----------

##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

In [ ]:
deuda_df.agg(F.mean('DT_AMT_MLAT_CD')).show()

+-------------------+
|avg(DT_AMT_MLAT_CD)|
+-------------------+
| 4.90062193498387E8|
+-------------------+



Eliminamos los valores nulos para que no causen problemas al calcular la media

In [ ]:
deuda_df_rm_na = deuda_df.dropna()

In [ ]:
deuda_df_rm_na.agg(F.mean('DT_DIS_BLAT_CD')).show()

+-------------------+
|avg(DT_DIS_BLAT_CD)|
+-------------------+
|     5.4106259761E9|
+-------------------+



In [ ]:
deuda_df_rm_na.agg(F.mean('DT_INT_BLAT_CD')).show()

+-------------------+
|avg(DT_INT_BLAT_CD)|
+-------------------+
|5.621520963000001E8|
+-------------------+



##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

In [108]:
from pyspark.sql.functions import col, avg, desc, sum
deuda_df.select("country_name", "DT_AMT_DLXF_CD").sort(desc("DT_AMT_DLXF_CD")).show(20)

+--------------------+----------------+
|        country_name|  DT_AMT_DLXF_CD|
+--------------------+----------------+
|               China|9.62186208357E10|
|              Brazil|9.00418403041E10|
|  Russian Federation|6.65897618335E10|
|              Turkey|5.15550310058E10|
|          South Asia|4.87562958982E10|
|               India|3.19235070008E10|
|           Indonesia|3.09161126538E10|
|          Kazakhstan|2.74820936864E10|
|              Mexico| 2.5218503927E10|
|Least developed c...|2.51970292994E10|
|            IDA only| 2.0483289208E10|
|             Romania|1.40137833504E10|
|            Colombia|1.19856744387E10|
|              Angola|1.10670456281E10|
|            Cameroon|1.04048149602E10|
|       Venezuela, RB|  9.8786592072E9|
|    Egypt, Arab Rep.|  9.6921141769E9|
|             Lebanon|  9.5069196696E9|
|        South Africa|  9.4742575519E9|
|             Vietnam|  8.8735059092E9|
+--------------------+----------------+
only showing top 20 rows



##### 6. Pais con los datos informados todos los años.

Vemos los distintos años que hay y el número de veces que aparecen.

In [110]:
desarrollo_df.groupBy("anho").count().show()

+-------------+-----+
|         anho|count|
+-------------+-----+
|2016 [YR2016]|   39|
|2014 [YR2014]|   50|
|2017 [YR2017]|    1|
|2015 [YR2015]|   44|
+-------------+-----+



A continuación debemos encontrar si hay algún país que esté en los 4 años

In [118]:
desarrollo_df.groupBy("country_name").count() \
    .withColumnRenamed("count", "conteo") \
    .filter("conteo == 4") \
    .show()

+------------+------+
|country_name|conteo|
+------------+------+
|      Malawi|     4|
+------------+------+



##### 7. Media anual de los distintos indicadores de desarrollo

In [ ]:
desarrollo_df.groupby('anho').agg(F.mean('GC_DOD_TOTL_CN')).show()

+-------------+--------------------+
|         anho| avg(GC_DOD_TOTL_CN)|
+-------------+--------------------+
|2016 [YR2016]|1.588138215480659...|
|2014 [YR2014]|1.034427587329532...|
|2017 [YR2017]| 2.23985861996269E12|
|2015 [YR2015]|1.318244039159025E14|
+-------------+--------------------+



In [ ]:
desarrollo_df.groupby('anho').agg(F.mean('GC_DOD_TOTL_GD_ZS')).show()

+-------------+----------------------+
|         anho|avg(GC_DOD_TOTL_GD_ZS)|
+-------------+----------------------+
|2016 [YR2016]|    58.796949624008356|
|2014 [YR2014]|    56.843914344384395|
|2017 [YR2017]|       48.659564238322|
|2015 [YR2015]|     58.36559834488034|
+-------------+----------------------+



In [ ]:
desarrollo_df.groupby('anho').agg(F.mean('SP_DYN_CBRT_IN')).show()

+-------------+-------------------+
|         anho|avg(SP_DYN_CBRT_IN)|
+-------------+-------------------+
|2016 [YR2016]| 16.144333333333332|
|2014 [YR2014]| 17.582620000000002|
|2017 [YR2017]|             34.593|
|2015 [YR2015]| 16.798477272727272|
+-------------+-------------------+



##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

Primero unimos ambos dataframes

In [120]:
join = desarrollo_df.join(other=deuda_df, on=["country_code", "country_name"], how="inner")

Seleccionamos solo las columnas que nos interesan.

In [122]:
join_select = join.select("country_name", "DT_AMT_MLAT_CD", "SP_DYN_CBRT_IN")
join_select.show()

+---------------+--------------+------------------+
|   country_name|DT_AMT_MLAT_CD|    SP_DYN_CBRT_IN|
+---------------+--------------+------------------+
|        Jamaica|  1.74427648E8|            16.814|
|        Jamaica|  1.74427648E8|            16.633|
|        Jamaica|  1.74427648E8|             16.46|
|Kyrgyz Republic|  7.52780372E7|              27.7|
|Kyrgyz Republic|  7.52780372E7|              27.4|
|Kyrgyz Republic|  7.52780372E7|              26.0|
|       Colombia|1.1009075536E9|            15.725|
|       Colombia|1.1009075536E9|            15.515|
|       Colombia|1.1009075536E9|            15.308|
|    Philippines| 7.390564273E8|            22.935|
|      Sri Lanka| 4.497719611E8|            16.849|
|      Sri Lanka| 4.497719611E8|            16.609|
|        Albania| 1.821976167E8|            12.259|
|        Albania| 1.821976167E8|12.197000000000001|
|        Albania| 1.821976167E8|             12.08|
|         Brazil|2.5143187416E9|            14.624|
|         Br

Agrupamos por país y deuda acumulada para que no haya repetidos y calculamos la media de la tasa de nacimiento.

In [125]:
join_select_grouped = join_select.groupBy("country_name", "DT_AMT_MLAT_CD").agg(avg("SP_DYN_CBRT_IN"))
join_select_grouped.show()

+--------------------+--------------+-------------------+
|        country_name|DT_AMT_MLAT_CD|avg(SP_DYN_CBRT_IN)|
+--------------------+--------------+-------------------+
|     Kyrgyz Republic|  7.52780372E7|  27.03333333333333|
|         El Salvador| 2.989396871E8| 18.733999999999998|
|    Papua New Guinea|   7.8345326E7|            28.1685|
|              Bhutan|  1.73904205E7|             17.791|
|             Albania| 1.821976167E8| 12.178666666666667|
|             Ukraine|1.0724728303E9|               10.6|
|              Jordan| 2.704987155E8|             24.305|
|              Turkey| 2.932698678E9| 16.717333333333332|
|         Philippines| 7.390564273E8|             22.935|
|           St. Lucia|  1.50192163E7|             12.502|
|           Sri Lanka| 4.497719611E8|             16.729|
|             Belarus|  5.95438826E8| 12.466666666666667|
|St. Vincent and t...|  2.16156007E7|             15.011|
|     Solomon Islands|     3952828.9| 33.525666666666666|
|             

Ordenamos por tasa de natalidad de menor a mayor y nos quedamos solo con los 10 primeros.

In [126]:
join_select_grouped = join_select_grouped.sort("avg(SP_DYN_CBRT_IN)").limit(10)
join_select_grouped.show()

+--------------------+--------------+-------------------+
|        country_name|DT_AMT_MLAT_CD|avg(SP_DYN_CBRT_IN)|
+--------------------+--------------+-------------------+
|             Ukraine|1.0724728303E9|               10.6|
|             Moldova| 1.358428655E8| 10.604333333333335|
|            Thailand|  3.90715715E7|            10.9175|
|             Albania| 1.821976167E8| 12.178666666666667|
|             Belarus|  5.95438826E8| 12.466666666666667|
|           St. Lucia|  1.50192163E7|             12.502|
|  Russian Federation|    1.416695E8| 13.166666666666666|
|             Georgia| 1.440624015E8|             14.056|
|              Brazil|2.5143187416E9| 14.467666666666668|
|St. Vincent and t...|  2.16156007E7|             15.011|
+--------------------+--------------+-------------------+



Sumamos la deuda acumulada de los anteriores y obtenemos la respuesta final.

In [137]:
join_select_grouped.select(F.sum("DT_AMT_MLAT_CD")).show()

+-------------------+
|sum(DT_AMT_MLAT_CD)|
+-------------------+
|4.861709170099999E9|
+-------------------+



##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?